## We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

In [27]:
import torch
import torch.nn as nn
import numpy as np

In [28]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 87, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [29]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

## Load Dataset

In [30]:
from torch.utils.data import TensorDataset

In [31]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [32]:
from torch.utils.data import DataLoader

In [33]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
#DataLoader, which can split the data into batches of a predefined size while training.
#It also provides other utilities like shuffling and random sampling of the data.

In [34]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break
#we have dataset in batches

tensor([[ 87., 135.,  57.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.],
        [ 92.,  87.,  64.]])
tensor([[118., 134.],
        [119., 133.],
        [ 56.,  70.],
        [ 20.,  38.],
        [ 82., 100.]])


In [35]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4094,  0.1716,  0.4452],
        [ 0.4077, -0.5723,  0.4341]], requires_grad=True)
Parameter containing:
tensor([-0.2687,  0.0470], requires_grad=True)


In [36]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.4094,  0.1716,  0.4452],
         [ 0.4077, -0.5723,  0.4341]], requires_grad=True),
 Parameter containing:
 tensor([-0.2687,  0.0470], requires_grad=True)]

In [37]:
# Generate predictions
preds = model(inputs)
preds

tensor([[  0.4864,  10.1341],
        [  6.0703,  14.5711],
        [ 12.9315, -15.9890],
        [-18.1771,  33.0881],
        [ 19.1218,   3.6276],
        [ -0.0947,  11.1141],
        [  6.3439,  15.5775],
        [ 12.9673, -15.1472],
        [-17.5961,  32.1081],
        [ 19.9764,   3.6540],
        [  0.7599,  11.1404],
        [  5.4892,  15.5511],
        [ 12.6580, -16.9954],
        [-19.0318,  33.0618],
        [ 19.7028,   2.6476]], grad_fn=<AddmmBackward0>)

In [38]:
# Import nn.functional
import torch.nn.functional as F

In [39]:
# Define loss function
loss_fn = F.mse_loss

In [40]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(7483.5791, grad_fn=<MseLossBackward0>)


## Optimizers

In [41]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [42]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):

    # Repeat for given number of epochs
    for epoch in range(num_epochs):

        # Train with batches of data
        for xb,yb in train_dl:

            # 1. Generate predictions
            pred = model(xb)

            # 2. Calculate loss
            loss = loss_fn(pred, yb)

            # 3. Compute gradients
            loss.backward()

            # 4. Update parameters using gradients
            opt.step()

            # 5. Reset the gradients to zero
            opt.zero_grad()

        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [43]:
fit(1000, model, loss_fn, opt, train_dl)

Epoch [10/1000], Loss: 511.7119
Epoch [20/1000], Loss: 548.3265
Epoch [30/1000], Loss: 574.0499
Epoch [40/1000], Loss: 153.9457
Epoch [50/1000], Loss: 142.3598
Epoch [60/1000], Loss: 95.2136
Epoch [70/1000], Loss: 79.4229
Epoch [80/1000], Loss: 26.0790
Epoch [90/1000], Loss: 34.5623


Epoch [100/1000], Loss: 16.1648
Epoch [110/1000], Loss: 11.6952
Epoch [120/1000], Loss: 8.0737
Epoch [130/1000], Loss: 5.5539
Epoch [140/1000], Loss: 2.2535
Epoch [150/1000], Loss: 4.3126
Epoch [160/1000], Loss: 1.7763
Epoch [170/1000], Loss: 2.8700
Epoch [180/1000], Loss: 3.2735
Epoch [190/1000], Loss: 1.6414
Epoch [200/1000], Loss: 1.7791
Epoch [210/1000], Loss: 1.6418
Epoch [220/1000], Loss: 1.4647
Epoch [230/1000], Loss: 0.9439
Epoch [240/1000], Loss: 1.4443
Epoch [250/1000], Loss: 1.6344
Epoch [260/1000], Loss: 1.6095
Epoch [270/1000], Loss: 1.0571
Epoch [280/1000], Loss: 1.5361
Epoch [290/1000], Loss: 1.4561
Epoch [300/1000], Loss: 1.3993
Epoch [310/1000], Loss: 0.9345
Epoch [320/1000], Loss: 1.2575
Epoch [330/1000], Loss: 1.3861
Epoch [340/1000], Loss: 1.0559
Epoch [350/1000], Loss: 1.4033
Epoch [360/1000], Loss: 1.3554
Epoch [370/1000], Loss: 1.4072
Epoch [380/1000], Loss: 1.3358
Epoch [390/1000], Loss: 1.3680
Epoch [400/1000], Loss: 1.0489
Epoch [410/1000], Loss: 0.8833
Epoch 

In [44]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 56.8826,  70.4219],
        [ 82.0400, 100.5342],
        [118.1464, 132.8668],
        [ 20.7965,  37.7825],
        [101.8436, 118.4779],
        [ 55.6384,  69.3427],
        [ 81.9004, 100.6259],
        [118.4454, 133.4664],
        [ 22.0408,  38.8617],
        [102.9482, 119.6488],
        [ 56.7430,  70.5136],
        [ 80.7958,  99.4550],
        [118.2861, 132.7750],
        [ 19.6920,  36.6116],
        [103.0879, 119.5570]], grad_fn=<AddmmBackward0>)

In [45]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [46]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.4131, 67.5618]], grad_fn=<AddmmBackward0>)

### The predicted yield of apples is 53.57 tons per hectare, and that of oranges is 67.54 tons per hectare.